In [ ]:
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow.keras.backend as K
from tensorflow import keras
from sklearn import model_selection
from sklearn.metrics import classification_report
import imgaug.augmenters as iaa
import imgaug as ia

In [ ]:
#image augmentation methods
ia.seed(1)
def Data_Aug(samples):
    randomaug = iaa.Sometimes(
        0.5,
        iaa.OneOf([
            iaa.Sequential([
                iaa.AllChannelsCLAHE(clip_limit=(1, 10)),
                iaa.Affine(rotate=(-5, 5))
            ]),
            iaa.Sequential([
                iaa.Affine(translate_percent={
                    "x": (-0.2, 0.2),
                    "y": (-0.2, 0.2)
                }),
                iaa.Multiply((0.7, 1.3)),
                iaa.AdditiveLaplaceNoise(scale=0.03 * 255, per_channel=True),
                iaa.Fliplr(0.3),
                iaa.Flipud(0.3)
            ]),
            iaa.Sequential([
                iaa.Affine(rotate=(-10, 10)),
                iaa.Multiply((0.7, 1.3)),
                iaa.AdditiveLaplaceNoise(scale=0.03 * 255, per_channel=True),
                iaa.Fliplr(0.3),
                iaa.Flipud(0.3)
            ]),
            iaa.Sequential([
                iaa.PerspectiveTransform(scale=(0, 0.15)),
                iaa.Multiply((0.7, 1.3)),
                iaa.AdditiveLaplaceNoise(scale=0.03 * 255, per_channel=True),
                iaa.Fliplr(0.3),
                iaa.Flipud(0.3)
            ])
        ]))


    return randomaug.augment_images(samples)

def resize_images(imgs, hight, width):
    seq = iaa.Resize({"height": hight, "width": width})
    aug = seq.augment_images(imgs)
    return aug

In [ ]:
class cifar100data(keras.utils.Sequence):
    '''
    generate data batch of cifar10
    10 classes
    '''
    def __init__(self, data_x, data_y, batch_size, train):
        self.data_x = data_x
        self.data_y = data_y
        self.train = train
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.data_y) / self.batch_size))

    def __getitem__(self, idx):

        batch_x = self.data_x[idx * self.batch_size:(idx + 1) *
                              self.batch_size]
        batch_y = self.data_y[idx * self.batch_size:(idx + 1) *
                              self.batch_size]
        if self.train == True:
            batch_x = Data_Aug(batch_x)

        batch_x = np.clip(batch_x / 255.0, 0, 1).astype("float32")
        batch_y = keras.utils.to_categorical(np.array(batch_y), 100)
        return batch_x, [batch_y, batch_y, batch_y, batch_y, batch_y]

In [ ]:
#callback for saving best model
class CustomEarlyStoppingAndSave(keras.callbacks.Callback):
    def __init__(self, save_path, patience=0):
        super(CustomEarlyStoppingAndSave, self).__init__()
        self.patience = patience
        self.save_path = save_path
        self.best_weights = None

    def on_train_begin(self, logs=None):
        self.wait = 0
        self.stopped_epoch = 0
        #self.best_val_distance_loss = np.Inf
        self.best_val_acc = 0

    def on_epoch_end(self, epoch, logs=None):
        #val_distance_loss = logs.get('val_MMD_loss')
        val_acc = logs.get('val_model_accuracy') + logs.get(
            'val_model_1_accuracy') + logs.get(
                'val_model_2_accuracy') + logs.get(
                    'val_model_3_accuracy') + logs.get('val_model_4_accuracy')

        if  np.greater(val_acc, self.best_val_acc):
            #self.best_val_distance_loss = val_distance_loss
            self.best_val_acc = val_acc
            self.wait = 0
            # Record the best weights if current results is better (less).
            self.best_weights = self.model.get_weights()
            self.stopped_epoch = epoch
        #else:
            #self.wait += 1
            #if self.wait >= self.patience:
                #self.stopped_epoch = epoch
                #self.model.stop_training = True
                #print("Restoring model weights from the end of the best epoch.")
                #self.model.set_weights(self.best_weights)

    def on_train_end(self, logs=None):
        self.model.set_weights(self.best_weights)
        if self.stopped_epoch > 0:
            print("Epoch %05d: early stopping and save the model" %
                  (self.stopped_epoch + 1))
        self.model.save(self.save_path)

In [ ]:
#Cosine&Euclidean distance loss
def descriptor(X, mask = False, max_norm = False):
  X = tf.reduce_mean(X, axis=-1)
  if mask:
    mean_X = tf.reduce_mean(X, axis = [1,2], keepdims= True)
    X = tf.where(X > mean_X, X , tf.zeros_like(X, dtype=tf.float32))
  if max_norm:
    max_X = tf.reduce_max(X, axis = [1,2], keepdims= True)
    X = tf.math.divide_no_nan(X, max_X)
  return X 
  
def compute_euclidean_sum(X, Y):
  L2_distance = tf.reduce_mean(tf.exp(-(X - Y)**2), axis= [1,2])
  return tf.reduce_mean(L2_distance)

def compute_cosine_sum(X, Y):
  X = tf.reshape(X, [tf.shape(X)[0], -1])
  Y = tf.reshape(Y, [tf.shape(Y)[0], -1])
  loss = -keras.losses.cosine_similarity(X,Y,axis=-1)
  return tf.reduce_mean(loss)

def cosine_euclidean_sum_loss(x, cosine_weight, euclidean_weight, mask, max_norm):
  descriptor0 = descriptor(x[0], mask=mask, max_norm=max_norm)
  descriptor1 = descriptor(x[1], mask=mask, max_norm=max_norm)
  descriptor2 = descriptor(x[2], mask=mask, max_norm=max_norm)
  descriptor3 = descriptor(x[3], mask=mask, max_norm=max_norm)
  descriptor4 = descriptor(x[4], mask=mask, max_norm=max_norm)
  cosine_sum = compute_cosine_sum(descriptor0, descriptor1)+compute_cosine_sum(descriptor0, descriptor2)+compute_cosine_sum(descriptor0, descriptor3)+compute_cosine_sum(descriptor0, descriptor4)+compute_cosine_sum(descriptor1, descriptor2)+compute_cosine_sum(descriptor1, descriptor3)+compute_cosine_sum(descriptor1, descriptor4)+compute_cosine_sum(descriptor2, descriptor3)+compute_cosine_sum(descriptor2, descriptor4)+compute_cosine_sum(descriptor3, descriptor4)
  euclidean_sum = compute_euclidean_sum(descriptor0, descriptor1)+compute_euclidean_sum(descriptor0, descriptor2)+compute_euclidean_sum(descriptor0, descriptor3)+compute_euclidean_sum(descriptor0, descriptor4)+compute_euclidean_sum(descriptor1, descriptor2)+compute_euclidean_sum(descriptor1, descriptor3)+compute_euclidean_sum(descriptor1, descriptor4)+compute_euclidean_sum(descriptor2, descriptor3)+compute_euclidean_sum(descriptor2, descriptor4)+compute_euclidean_sum(descriptor3, descriptor4)
  return cosine_sum, euclidean_sum, cosine_weight*cosine_sum+euclidean_weight*euclidean_sum  

In [ ]:
#create single ResNet model
def creat_ResNet(input_shape, num_classes=100, num_filters = 64, random_seed=None):
  inputs = keras.Input(shape=input_shape)
  y = keras.layers.Conv2D(num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(inputs)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  y = keras.layers.Conv2D(num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(y)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  y = keras.layers.Conv2D(num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(y)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  x = keras.layers.Conv2D(num_filters, (1, 1), kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(inputs)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.add([x, y])
  x = keras.layers.LeakyReLU(alpha=0.1)(x)
  x = keras.layers.MaxPooling2D((2,2))(x)

  y = keras.layers.Conv2D(2*num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  y = keras.layers.Conv2D(2*num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(y)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  y = keras.layers.Conv2D(2*num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(y)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  x = keras.layers.Conv2D(2*num_filters, (1, 1), kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.add([x, y])
  x = keras.layers.LeakyReLU(alpha=0.1)(x)
  x = keras.layers.MaxPooling2D((2,2))(x) 

  y = keras.layers.Conv2D(4*num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  y = keras.layers.Conv2D(4*num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(y)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  y = keras.layers.Conv2D(4*num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(y)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  x = keras.layers.Conv2D(4*num_filters, (1, 1), kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.add([x, y])
  x = keras.layers.LeakyReLU(alpha=0.1)(x)
  last_acti = keras.layers.MaxPooling2D((2,2))(x) 

  '''  
  y = keras.layers.Conv2D(8*num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  y = keras.layers.Conv2D(8*num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(y)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  y = keras.layers.Conv2D(8*num_filters, (3, 3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(y)
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU(alpha=0.1)(y)
  x = keras.layers.Conv2D(8*num_filters, (1, 1), kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.add([x, y])
  x = keras.layers.LeakyReLU(alpha=0.1)(x)
  last_acti = keras.layers.MaxPooling2D((2,2))(x) 
  '''

  x = keras.layers.GlobalAveragePooling2D()(last_acti)
  outputs = keras.layers.Dense(num_classes, activation='softmax', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x)
  model = keras.Model(inputs=inputs, outputs=[outputs,last_acti])
  return model

In [ ]:
#construct joint training model for the five base models
input_size = (32, 32, 3)
model0 = creat_ResNet(input_shape=input_size, num_classes=100, num_filters = 64, random_seed=1)#random_seed: initialization
model1 = creat_ResNet(input_shape=input_size, num_classes=100, num_filters = 64, random_seed=2)
model2 = creat_ResNet(input_shape=input_size, num_classes=100, num_filters = 64, random_seed=3)
model3 = creat_ResNet(input_shape=input_size, num_classes=100, num_filters = 64, random_seed=4)
model4 = creat_ResNet(input_shape=input_size, num_classes=100, num_filters = 64, random_seed=5)
allinputs = keras.Input(shape=input_size)
output0, feature_maps0 = model0(allinputs)
output1, feature_maps1 = model1(allinputs)
output2, feature_maps2 = model2(allinputs)
output3, feature_maps3 = model3(allinputs)
output4, feature_maps4 = model4(allinputs)
model_train = keras.Model(
    inputs=allinputs, outputs=[output0, output1, output2, output3, output4])

#implement distance loss
loss1, loss2, loss = cosine_euclidean_sum_loss(x=[feature_maps0, feature_maps1, feature_maps2, feature_maps3, feature_maps4], 
                                      cosine_weight=1, euclidean_weight=10, mask=True, max_norm=False)


model_train.add_loss(loss)
model_train.add_metric(loss1, name="loss1", aggregation='mean')
model_train.add_metric(loss2, name="loss2", aggregation='mean')

model_train.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),
                    loss=['categorical_crossentropy','categorical_crossentropy','categorical_crossentropy','categorical_crossentropy','categorical_crossentropy'],
                    metrics=['accuracy'])

#load Cifar100 data
(x_origin_train, y_origin_train), (x_test,y_test) = keras.datasets.cifar100.load_data()
y_origin_train = y_origin_train.reshape(-1,)
y_test = y_test.reshape(-1,)

x_train, x_val, y_train, y_val = model_selection.train_test_split(
    x_origin_train,
    y_origin_train,
    test_size=10000,
    random_state=0,
    stratify=y_origin_train)
_, x_val, _, y_val = model_selection.train_test_split(
    x_val,
    y_val,
    test_size=2000,
    random_state=0,
    stratify=y_val)
#select training samples
train_index = []
random.seed(0)
for i in range(100):
    train_index += (random.sample(list(np.where(y_train == i)[0]), 3))
random.shuffle(train_index)
sampled_x_train = x_train[train_index]
sampled_y_train = y_train[train_index]
del train_index

#construct training and validation samples
train_sequence = cifar100data(sampled_x_train, sampled_y_train, batch_size=10, train=True)
validation_sequence = cifar100data(x_val, y_val, batch_size=10, train=False)

#training
history = model_train.fit(
    train_sequence,
    epochs=300,
    validation_data=validation_sequence,
    #validation_steps=200,
    callbacks=CustomEarlyStoppingAndSave(
        save_path='100_samples_seed_fix.h5',
        patience=10))
frame = pd.DataFrame({
    'loss':
    history.history['loss'],
    'loss1':
    history.history['loss1'],
    'loss2':
    history.history['loss2'],
    'model_0_loss':
    history.history['model_loss'],
    'model_1_loss':
    history.history['model_1_loss'],
    'model_2_loss':
    history.history['model_2_loss'],
    'model_3_loss':
    history.history['model_3_loss'],
    'model_4_loss':
    history.history['model_4_loss'],
    'model_0_accuracy':
    history.history['model_accuracy'],
    'model_1_accuracy':
    history.history['model_1_accuracy'],
    'model_2_accuracy':
    history.history['model_2_accuracy'],
    'model_3_accuracy':
    history.history['model_3_accuracy'],
    'model_4_accuracy':
    history.history['model_4_accuracy'],
    'val_loss':
    history.history['val_loss'],
    'val_loss1':
    history.history['val_loss1'],
    'val_loss2':
    history.history['val_loss2'],
    'val_model_0_loss':
    history.history['val_model_loss'],
    'val_model_1_loss':
    history.history['val_model_1_loss'],
    'val_model_2_loss':
    history.history['val_model_2_loss'],
    'val_model_3_loss':
    history.history['val_model_3_loss'],
    'val_model_4_loss':
    history.history['val_model_4_loss'],
    'val_model_0_accuracy':
    history.history['val_model_accuracy'],
    'val_model_1_accuracy':
    history.history['val_model_1_accuracy'],
    'val_model_2_accuracy':
    history.history['val_model_2_accuracy'],
    'val_model_3_accuracy':
    history.history['val_model_3_accuracy'],
    'val_model_4_accuracy':
    history.history['val_model_4_accuracy'],
})
#frame.to_excel('/content/drive/MyDrive/cifar100experiment/resnet/5models300s/table_400_samples_seed_fix.xlsx')


Epoch 1/300
30/30 [==============================] - 28s 211ms/step - loss: 83.9790 - model_loss: 5.9373 - model_1_loss: 6.2301 - model_2_loss: 6.2126 - model_3_loss: 6.1664 - model_4_loss: 6.3492 - model_accuracy: 0.0033 - model_1_accuracy: 0.0000e+00 - model_2_accuracy: 0.0033 - model_3_accuracy: 0.0167 - model_4_accuracy: 0.0033 - loss1: 2.9638 - loss2: 5.0120 - val_loss: 99.8942 - val_model_loss: 6.8230 - val_model_1_loss: 6.1913 - val_model_2_loss: 15.0424 - val_model_3_loss: 5.0919 - val_model_4_loss: 23.3291 - val_model_accuracy: 0.0150 - val_model_1_accuracy: 0.0110 - val_model_2_accuracy: 0.0100 - val_model_3_accuracy: 0.0115 - val_model_4_accuracy: 0.0100 - val_loss1: 2.1869 - val_loss2: 4.1230
Epoch 2/300
30/30 [==============================] - 4s 143ms/step - loss: 69.1531 - model_loss: 4.5995 - model_1_loss: 4.7017 - model_2_loss: 4.6248 - model_3_loss: 4.6901 - model_4_loss: 4.7299 - model_accuracy: 0.0300 - model_1_accuracy: 0.0200 - model_2_accuracy: 0.0500 - model_3_a

In [ ]:
#evaluation of the base models
x_test = x_test/255.0
evaluate = model_train.evaluate(x_test,[keras.utils.to_categorical(y_test),keras.utils.to_categorical(y_test),keras.utils.to_categorical(y_test),keras.utils.to_categorical(y_test),keras.utils.to_categorical(y_test)],batch_size=10)
evaluate

1000/1000 [==============================] - 16s 14ms/step - loss: 105.0552 - model_loss: 10.0921 - model_1_loss: 9.2972 - model_2_loss: 9.4439 - model_3_loss: 9.8721 - model_4_loss: 10.9657 - model_accuracy: 0.0802 - model_1_accuracy: 0.0836 - model_2_accuracy: 0.0894 - model_3_accuracy: 0.0781 - model_4_accuracy: 0.0807 - loss1: 0.3291 - loss2: 5.5055


In [ ]:
#hard voting ensemble
class HardVotingEnsemble:
    def __init__(self, model, testdata ,batch_size):
        self.ensemble_model = model
        self.testdata = testdata
        self.batch_size = batch_size
        self.data_size = testdata[1].shape[0]
    def prediction(self):
        predictions =[]
        for i in range(int(self.data_size/self.batch_size)):
            batch_predictions = np.argmax(self.ensemble_model.predict(self.testdata[0][i*self.batch_size:(i+1)*self.batch_size]),axis=2) 
            batch_predictions = np.stack(batch_predictions, axis=1)
            batch_predictions = [np.argmax(np.bincount(pre)) for pre in batch_predictions]
            predictions = predictions+batch_predictions
        return np.array(predictions)

    def evaluate(self):
        predictions = self.prediction()
        accuracy = np.sum(
            predictions == self.testdata[1]) / self.data_size
        return accuracy
HEnsemble = HardVotingEnsemble(model = model_train,testdata = (x_test, y_test),batch_size=10)
HEnsemble.evaluate()

0.0973

In [ ]:
#soft voting ensemble
def SoftVotingEnsemble(model, x, y):
  data_size = x.shape[0]
  predictions = tf.argmax(tf.reduce_mean(model.predict(x), axis = 0),axis=1)
  accuracy = np.sum(
     predictions == y) / data_size
  return accuracy
SoftVotingEnsemble(model_train, x_test, y_test)

0.0996

In [ ]:
#callback for ensemble learning
class eCustomEarlyStoppingAndSave(keras.callbacks.Callback):
    def __init__(self, save_path, patience=0):
        super(eCustomEarlyStoppingAndSave, self).__init__()
        self.patience = patience
        self.save_path = save_path
        self.best_weights = None

    def on_train_begin(self, logs=None):
        #self.wait = 0
        self.stopped_epoch = 0
        #self.best_val_distance_loss = np.Inf
        self.best_val_acc = 0

    def on_epoch_end(self, epoch, logs=None):
        #val_distance_loss = logs.get('val_MMD_loss')
        val_acc = logs.get('val_accuracy') 

        if  np.greater(val_acc, self.best_val_acc):
            #self.best_val_distance_loss = val_distance_loss
            self.best_val_acc = val_acc
            #self.wait = 0
            # Record the best weights if current results is better (less).
            self.best_weights = self.model.get_weights()
            self.stopped_epoch = epoch
        #else:
            #self.wait += 1
            #if self.wait >= self.patience:
                #self.stopped_epoch = epoch
                #self.model.stop_training = True
                #print("Restoring model weights from the end of the best epoch.")
                #self.model.set_weights(self.best_weights)

    def on_train_end(self, logs=None):
        self.model.set_weights(self.best_weights)
        if self.stopped_epoch >= 0:
            print("Epoch %05d: early stopping and save the model" %
                  (self.stopped_epoch + 1))
        self.model.save(self.save_path)

In [ ]:
class ecifar100data(keras.utils.Sequence):
    '''
    generate data batch of cifar10
    10 classes
    '''
    def __init__(self, data_x, data_y, batch_size, train):
        self.data_x = data_x
        self.data_y = data_y
        self.train = train
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.data_y) / self.batch_size))

    def __getitem__(self, idx):

        batch_x = self.data_x[idx * self.batch_size:(idx + 1) *
                              self.batch_size]
        batch_y = self.data_y[idx * self.batch_size:(idx + 1) *
                              self.batch_size]
        if self.train == True:
            batch_x = Data_Aug(batch_x)

        batch_x = np.clip(batch_x / 255.0, 0, 1).astype("float32")
        batch_y = keras.utils.to_categorical(np.array(batch_y), 100)
        return batch_x, batch_y

In [ ]:
#feature fusion model
model_path = '100_samples_seed_fix.h5'
model = keras.models.load_model(model_path)
extractor_1 = keras.models.Model(inputs=model.layers[1].inputs,outputs=model.layers[1].layers[42].output)
extractor_2 = keras.models.Model(inputs=model.layers[2].inputs,outputs=model.layers[2].layers[42].output)
extractor_3 = keras.models.Model(inputs=model.layers[3].inputs,outputs=model.layers[3].layers[42].output)
extractor_4 = keras.models.Model(inputs=model.layers[4].inputs,outputs=model.layers[4].layers[42].output)
extractor_5 = keras.models.Model(inputs=model.layers[5].inputs,outputs=model.layers[5].layers[42].output)
extractor_1.trainable = False
extractor_2.trainable = False
extractor_3.trainable = False
extractor_4.trainable = False
extractor_5.trainable = False
inputs = keras.Input(shape = input_size)
features_1 = extractor_1(inputs, training = False)
features_2 = extractor_2(inputs, training = False)
features_3 = extractor_3(inputs, training = False)
features_4 = extractor_4(inputs, training = False)
features_5 = extractor_5(inputs, training = False)

x = keras.layers.Concatenate(axis=-1)([features_1,features_2,features_3,features_4,features_5])

x = keras.layers.Flatten()(x)
x = keras.layers.Dense(512,  name='dense_100')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Activation('relu')(x)
x = keras.layers.Dropout(0.5)(x)
outputs = keras.layers.Dense(100, activation='softmax', name='dense_101')(x)
ensemble = keras.Model(inputs = inputs, outputs = outputs)
ensemble.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-4),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])
train_sequence = ecifar100data(sampled_x_train, sampled_y_train, batch_size=10, train=True)
validation_sequence = ecifar100data(x_val, y_val, batch_size=10, train=False)
history = ensemble.fit(
    train_sequence,
    epochs=50,
    validation_data=validation_sequence,
    #validation_steps=200,
    callbacks=eCustomEarlyStoppingAndSave(
        save_path='ensemble.h5',
        patience=10))
evaluate = ensemble.evaluate(x_test,keras.utils.to_categorical(y_test),batch_size=10)

Epoch 1/50
30/30 [==============================] - 7s 114ms/step - loss: 5.2387 - accuracy: 0.0133 - val_loss: 6.8324 - val_accuracy: 0.0135
Epoch 2/50
30/30 [==============================] - 2s 77ms/step - loss: 4.5463 - accuracy: 0.0400 - val_loss: 5.5845 - val_accuracy: 0.0165
Epoch 3/50
30/30 [==============================] - 2s 79ms/step - loss: 3.8057 - accuracy: 0.0867 - val_loss: 5.0703 - val_accuracy: 0.0220
Epoch 4/50
30/30 [==============================] - 2s 77ms/step - loss: 3.2442 - accuracy: 0.2100 - val_loss: 4.8254 - val_accuracy: 0.0320
Epoch 5/50
30/30 [==============================] - 2s 77ms/step - loss: 2.7393 - accuracy: 0.3700 - val_loss: 4.6852 - val_accuracy: 0.0415
Epoch 6/50
30/30 [==============================] - 2s 78ms/step - loss: 2.3614 - accuracy: 0.4867 - val_loss: 4.5959 - val_accuracy: 0.0550
Epoch 7/50
30/30 [==============================] - 2s 76ms/step - loss: 1.9997 - accuracy: 0.6600 - val_loss: 4.5407 - val_accuracy: 0.0615
Epoch 8/50
3

In [ ]:
#finetuning
extractor_1.trainable = True
extractor_2.trainable = True
extractor_3.trainable = True
extractor_4.trainable = True
extractor_5.trainable = True
ensemble.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-5),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])
#train_sequence = ecifar10data(sampled_x_train, sampled_y_train, batch_size=10, train=False)
#validation_sequence = ecifar10data(x_val, y_val, batch_size=10, train=False)
history = ensemble.fit(
    train_sequence,
    epochs=20,
    validation_data=validation_sequence,
    #validation_steps=200,
    callbacks=eCustomEarlyStoppingAndSave(
        save_path='ensemble.h5',
        patience=10))
evaluate = ensemble.evaluate(x_test,keras.utils.to_categorical(y_test),batch_size=10)

Epoch 1/20
30/30 [==============================] - 10s 162ms/step - loss: 0.2854 - accuracy: 1.0000 - val_loss: 4.3348 - val_accuracy: 0.1010
Epoch 2/20
30/30 [==============================] - 3s 88ms/step - loss: 0.2310 - accuracy: 1.0000 - val_loss: 4.3424 - val_accuracy: 0.0980
Epoch 3/20
30/30 [==============================] - 3s 94ms/step - loss: 0.2078 - accuracy: 1.0000 - val_loss: 4.3455 - val_accuracy: 0.1030
Epoch 4/20
30/30 [==============================] - 3s 89ms/step - loss: 0.1704 - accuracy: 1.0000 - val_loss: 4.3421 - val_accuracy: 0.1020
Epoch 5/20
30/30 [==============================] - 3s 88ms/step - loss: 0.1553 - accuracy: 1.0000 - val_loss: 4.3416 - val_accuracy: 0.1025
Epoch 6/20
30/30 [==============================] - 3s 88ms/step - loss: 0.1347 - accuracy: 1.0000 - val_loss: 4.3460 - val_accuracy: 0.1000
Epoch 7/20
30/30 [==============================] - 3s 87ms/step - loss: 0.1253 - accuracy: 1.0000 - val_loss: 4.3434 - val_accuracy: 0.0995
Epoch 8/20


In [ ]:
#f1 score
predict = tf.argmax(ensemble.predict(x_test),axis=-1)
classification_report(y_test, predict,digits=4)

'              precision    recall  f1-score   support\n\n           0     0.4091    0.2700    0.3253       100\n           1     0.0682    0.0300    0.0417       100\n           2     0.0809    0.1100    0.0932       100\n           3     0.0000    0.0000    0.0000       100\n           4     0.0351    0.0200    0.0255       100\n           5     0.1000    0.0200    0.0333       100\n           6     0.0263    0.0500    0.0345       100\n           7     0.6000    0.0300    0.0571       100\n           8     0.1250    0.0100    0.0185       100\n           9     0.0635    0.0400    0.0491       100\n          10     0.0000    0.0000    0.0000       100\n          11     0.0000    0.0000    0.0000       100\n          12     0.0938    0.1200    0.1053       100\n          13     0.2115    0.1100    0.1447       100\n          14     0.1111    0.0300    0.0472       100\n          15     0.0190    0.0400    0.0258       100\n          16     0.0312    0.0200    0.0244       100\n       